In [1]:
import os 
import numpy as np 
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense , Dropout,Flatten,Rescaling
import pennylane as qml
from pennylane import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
import torch
import torch.nn as nn
import torch.nn.functional as F
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

In [2]:
dirs = os.listdir("Sampled_images/")
count = 0
for dir in dirs:
    files = list(os.listdir("Sampled_images/" + dir + "/")) 
    print( dir + " Folder has " + str(len(files))+ " Images")
    count = count + len(files)
    
print("Images folder has " + str(count) + " Images")  

MildDemented Folder has 1000 Images
ModerateDemented Folder has 1000 Images
NonDemented Folder has 1000 Images
VeryMildDemented Folder has 1000 Images
Images folder has 4000 Images


In [3]:
base_dir = "Sampled_images/"
img_size = 180
batch = 10 

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,  
    validation_split=0.2,  
    subset="training",  
    batch_size=batch,  
    image_size=(img_size, img_size)
    
) 

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    seed=123,  
    validation_split=0.2,  
    subset="validation",  
    batch_size=batch,  
    image_size=(img_size, img_size)
    
)

Found 4000 files belonging to 4 classes.
Using 3200 files for training.
Found 4000 files belonging to 4 classes.
Using 800 files for validation.


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit.primitives import Sampler

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from qiskit.circuit import ParameterVector, QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# === Quantum Circuit Function (6 qubits) ===
def create_multiclass_pqc(num_qubits=6):
    x = ParameterVector("x", num_qubits)
    theta = ParameterVector("θ", num_qubits)

    qc = QuantumCircuit(num_qubits)

    # Angle encoding
    for i in range(num_qubits):
        qc.h(i)
        qc.ry(x[i], i)

    # Entanglement
    for i in range(num_qubits - 1):
        qc.cz(i, i + 1)

    # Trainable weights
    for i in range(num_qubits):
        qc.ry(theta[i], i)

    # Measurement observables (Z per qubit)
    observables = [SparsePauliOp('I' * i + 'Z' + 'I' * (num_qubits - i - 1)) for i in range(num_qubits)]

    return qc, list(x), list(theta), observables

# === Multi-Class Classical-Quantum CNN (6 Qubits) ===
class MultiClassCQCNN(nn.Module):
    def __init__(self, num_qubits=6, num_classes=4):
        super(MultiClassCQCNN, self).__init__()
        self.num_qubits = num_qubits

        # Classical Feature Extractor for Grayscale (1-channel)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # 180x180 → 180x180
            nn.ReLU(),
            nn.MaxPool2d(2),  # → 90x90

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # → 90x90
            nn.ReLU(),
            nn.MaxPool2d(2),  # → 45x45

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # → 45x45
            nn.ReLU(),
            nn.MaxPool2d(5),  # → 9x9
        )

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 9 * 9, num_qubits)  # Reduce to quantum input size (6)

        # Quantum Circuit & QNN
        qc, input_params, weight_params, observables = create_multiclass_pqc(num_qubits)
        estimator = Estimator()
        qnn = EstimatorQNN(
            circuit=qc,
            input_params=input_params,
            weight_params=weight_params,
            observables=observables,
            estimator=estimator,
            input_gradients=True
        )

        # Torch-Quantum Bridge
        self.q_layer = TorchConnector(qnn)

        # Final classification layer
        self.classifier = nn.Linear(num_qubits, num_classes)  # 6 → 4 classes

    def forward(self, x):
        x = self.conv_layers(x)         # Classical CNN
        x = self.flatten(x)
        x = self.fc1(x)                 # To quantum input size (6 features)
        x = torch.tanh(x)               # Normalize input to range [-1, 1]
        x = self.q_layer(x)             # Quantum Layer (6 outputs)
        x = self.classifier(x)          # Output logits (4 classes for Alzheimer's stages)
        return x

In [10]:
class MultiClassCQCNN(nn.Module):
    def __init__(self, num_qubits=4, num_classes=4):
        super(MultiClassCQCNN, self).__init__()
        self.num_qubits = num_qubits

        # Classical Feature Extractor with Batch Normalization
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            
            nn.ReLU(),
            nn.MaxPool2d(5),
        )

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 9 * 9, num_qubits)

        # Quantum Circuit & QNN
        qc, input_params, weight_params, observables = create_multiclass_pqc(num_qubits)
        
        # Fixed: Use SamplerQNN for better stability
        sampler = Sampler()
        qnn = SamplerQNN(
            circuit=qc,
            input_params=input_params,
            weight_params=weight_params,
            sampler=sampler,
            input_gradients=True
        )

        # Torch-Quantum Bridge
        self.q_layer = TorchConnector(qnn)
        
        # Fixed: Initialize quantum weights
        self._init_quantum_weights()

        # Final classification layer
        self.classifier = nn.Linear(2**num_qubits, num_classes)  # SamplerQNN outputs 2^n values

    def _init_quantum_weights(self):
        """Initialize quantum layer weights"""
        with torch.no_grad():
            # Small random initialization for quantum parameters
            if hasattr(self.q_layer, 'weight') and self.q_layer.weight is not None:
                nn.init.uniform_(self.q_layer.weight, -0.1, 0.1)

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.fc1(x)
        # Fixed: Apply tanh first, then scale to [-π, π]
        x = torch.tanh(x) * torch.pi
        x = self.q_layer(x)
        x = self.classifier(x)
        return x


In [7]:
class MultiClassCQCNN(nn.Module):
    def __init__(self, num_qubits=4, num_classes=4):
        super(MultiClassCQCNN, self).__init__()
        self.num_qubits = num_qubits

        # Classical Feature Extractor for Grayscale (1-channel)
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),  # 180x180 → 180x180
            nn.ReLU(),
            nn.MaxPool2d(2),  # → 90x90

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),  # → 90x90
            nn.ReLU(),
            nn.MaxPool2d(2),  # → 45x45

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # → 45x45
            nn.ReLU(),
            nn.MaxPool2d(5),  # → 9x9
        )

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 9 * 9, num_qubits)  # Reduce to quantum input size

        # Quantum Circuit & QNN
        qc, input_params, weight_params, observables = create_multiclass_pqc(num_qubits)
        estimator = Estimator()
        qnn = EstimatorQNN(
            circuit=qc,
            input_params=input_params,
            weight_params=weight_params,
            observables=observables,
            estimator=estimator,
            input_gradients=True
        )

        # Torch-Quantum Bridge
        self.q_layer = TorchConnector(qnn)

        # Final classification layer
        self.classifier = nn.Linear(num_qubits, num_classes)

    def forward(self, x):
        x = self.conv_layers(x)         # Classical CNN
        x = self.flatten(x)
        x = self.fc1(x)                 # To quantum input size
        x = torch.tanh(x)               # Normalize input to range [-1, 1]
        x = self.q_layer(x)             # Quantum Layer
        x = self.classifier(x)          # Output logits
        return x

In [8]:
from qiskit.quantum_info.operators import SparsePauliOp

model = MultiClassCQCNN(num_qubits=6)
print(model)

MultiClassCQCNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=5184, out_features=6, bias=True)
  (q_layer): TorchConnector()
  (classifier): Linear(in_features=6, out_features=4, bias=True)
)


In [9]:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal trainable parameters: {total_params}")


Total trainable parameters: 54440


In [10]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MultiClassCQCNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=5184, out_features=6, bias=True)
  (q_layer): TorchConnector()
  (classifier): Linear(in_features=6, out_features=4, bias=True)
)

In [11]:
import torch.optim as optim
from copy import deepcopy

In [12]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [13]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# ✅ Define transform with grayscale conversion
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # ⬅️ Ensure grayscale
    transforms.Resize((180, 180)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# ✅ Load dataset
base_dir = "Sampled_images"
full_dataset = datasets.ImageFolder(root=base_dir, transform=transform)

# ✅ Train/Validation split
val_split = 0.2
val_size = int(len(full_dataset) * val_split)
train_size = len(full_dataset) - val_size

train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# ✅ Create DataLoaders
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [14]:
num_epochs = 100
patience = 5

best_val_loss = float('inf')
best_model_wts = None
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)  # logits output
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss = val_loss / val_total
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = deepcopy(model.state_dict())
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# Load best model weights after early stopping
if best_model_wts:
    model.load_state_dict(best_model_wts)

Epoch 1/100 | Train Loss: 1.3185 | Train Acc: 0.3416 | Val Loss: 1.1423 | Val Acc: 0.5238
Epoch 2/100 | Train Loss: 1.0158 | Train Acc: 0.5784 | Val Loss: 0.9066 | Val Acc: 0.6462
Epoch 3/100 | Train Loss: 0.8363 | Train Acc: 0.6444 | Val Loss: 0.9016 | Val Acc: 0.6012
Epoch 4/100 | Train Loss: 0.7482 | Train Acc: 0.6781 | Val Loss: 0.7436 | Val Acc: 0.6675
Epoch 5/100 | Train Loss: 0.6600 | Train Acc: 0.7191 | Val Loss: 0.7281 | Val Acc: 0.6887
Epoch 6/100 | Train Loss: 0.6011 | Train Acc: 0.7444 | Val Loss: 0.6546 | Val Acc: 0.7225
Epoch 7/100 | Train Loss: 0.5310 | Train Acc: 0.7822 | Val Loss: 0.6710 | Val Acc: 0.7163
Epoch 8/100 | Train Loss: 0.5051 | Train Acc: 0.8022 | Val Loss: 0.6559 | Val Acc: 0.7113
Epoch 9/100 | Train Loss: 0.4267 | Train Acc: 0.8391 | Val Loss: 0.6157 | Val Acc: 0.7388
Epoch 10/100 | Train Loss: 0.3707 | Train Acc: 0.8644 | Val Loss: 0.6401 | Val Acc: 0.7400
Epoch 11/100 | Train Loss: 0.3393 | Train Acc: 0.8800 | Val Loss: 0.5932 | Val Acc: 0.7688
Epoch 12